https://www.science.org/doi/10.1126/science.1127647

https://github.com/L1aoXingyu/pytorch-beginner/blob/9c86be785c7c318a09cf29112dd1f1a58613239b/08-AutoEncoder/conv_autoencoder.py

In [2]:
%load_ext autoreload
%autoreload 2

In [40]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch import nn
from pathlib import Path
from torch.utils.data import TensorDataset, DataLoader
from lib.modules import (
    evaluate_loop, 
    pad_for_windowing,
    window_session,
    optimization_loop
)
from tqdm import tqdm

In [9]:
WINSIZE = 101
DEVICE = 'cuda'
RAW_DIR = Path('/home/mmazeem/datasets/eating_raw/')

In [17]:
recordings = []
for rec_dir in RAW_DIR.iterdir():
    recordings.append(rec_dir)
    print(rec_dir.name)

andrew-2023-11-01_15_49_48
andrew-2023-10-26_15_32_20


In [18]:
session_dir = recordings[0]
acceleration = pd.read_csv(session_dir / 'acceleration.csv',skiprows=1)
acceleration_start_time_seconds = float(pd.read_csv(session_dir / 'acceleration.csv',nrows=1,header=None).iloc[0,0].split()[-1])/1000
acceleration.timestamp = ((acceleration.timestamp - acceleration.timestamp[0])*1e-9)+acceleration_start_time_seconds
acceleration = acceleration.dropna()

In [27]:
Xte = torch.Tensor(acceleration[['x','y','z']].values)
Xte = pad_for_windowing(Xte, WINSIZE)
Xte = window_session(Xte, WINSIZE)
trainloader = DataLoader(TensorDataset(Xte), batch_size=64, shuffle=True)

In [35]:
# First: Autoencoder reduces input to 3 dimensions

class Autoencoder(nn.Module):
    def __init__(self, winsize):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(winsize*3, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(), 
            nn.Linear(64, 12), 
            nn.ReLU(True), 
            nn.Linear(12, 3)
        )
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(),
            nn.Linear(12, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(), 
            nn.Linear(128, winsize*3)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
model = Autoencoder(winsize=WINSIZE).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
criterion = nn.BCEWithLogitsLoss()

In [41]:
losses = []
pbar = tqdm(range(10))
for epoch in pbar:
    lossi = 0
    for X in trainloader:
        X = X[0].to(DEVICE)

        logits = model(X)
        loss = criterion(logits, X)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        lossi += loss.item()

    losses.append(loss/len(trainloader))
    pbar.set_description(f'Epoch {epoch}: Train Loss: {losses[-1]:.5}')

plt.plot(losses)

  0%|          | 0/10 [00:00<?, ?it/s]